#  Chapter 1 DataCleaning and Exploratory Data Analysis

This first session deals with data cleaning, EDA and builing a simplistic model that can be used for predicting the sale price of houses in Ames, IOWA. Here we try to understand the dataset and get familiar with the methodology for developing an efficient model.

The next session contains a report that contains  the detailed analysis, interpretation, and information for the models. We will do a cross-validation and data splitting where the original data set is split into two data sets: the training set and the validation set. 



## The Data
There are two data sets included in the data folder: `Ames_Housing_Price_Data.csv` and `Ames_Real_Estate_Data.csv`.

The `Ames_Housing_Price_Data.csv` set contains $81$ data columns, including the key feature **SalePrice** which will be used as the target of the predictive/descriptive modeling. **PID** refers to the land parcel ID, which can merged on the *MapRefNo* column of the **Ames Accessor Data** (`Ames_Real_Estate_Data.csv`) to find the property address. Using a free service, such as **geopy**, we can find the long-lat coordinates of the houses.

The columns of the data are mostly attributes associated with the land and the houses. There are size related attributes, quality and condition attributes, house attachment attributes, etc.

To establish a foundation for your team's data analytics, we offer some insights on the house sizes vs. prices.

## Outliers

The instructors' notes state:

> **Five observations** that an instructor may wish to remove from the data set before giving it to students (a plot of SALE PRICE versus GR LIV AREA will quickly indicate these
points). Three of them are true **outliers** (Partial Sales that likely don’t represent actual market values) and two of them are simply unusual sales (very large houses priced
relatively appropriately). I would **recommend removing any houses with more than
3700 square feet** from the data set (which eliminates these five unusual observations)
before assigning it to students.

To apply a more "rigorous" approach, outlier detection is conducted with a so-called Isolation Forest.

In [ ]:
# Import the datasets and the libraries
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statistics as stats
realEstate = pd.read_csv("Ames_Real_Estate_Data.csv")
realEstate = realEstate[['MapRefNo','Prop_Addr','MA_Zip1']]
geocode_data =pd.read_csv("geocode_data.csv")

#Linear Models
from sklearn.linear_model import Lasso, ElasticNet


#Kernel Ridge Regression
from sklearn.kernel_ridge import KernelRidge


#Gradient Boosting Machines
from sklearn.ensemble import GradientBoostingRegressor

#Random Forest
from sklearn.ensemble import RandomForestRegressor

#Support Vector Machines
from sklearn import svm

#Linear Regression
from sklearn import linear_model

#Make Pipeline
from sklearn.pipeline import make_pipeline

#Robust Scaler
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split, KFold, cross_val_score


housing = pd.read_csv('Ames_HousePrice.csv', index_col=0)
housing = housing[housing.GrLivArea<3700]
from scipy import stats
housing.head()

In [ ]:
year = housing.groupby("YrSold").mean()
plt.scatter(year[['GrLivArea']], year["SalePrice"])
year
list(map(lambda x,y: x/y, housing.groupby("YrSold")["SalePrice"].mean(), housing.groupby("YrSold")["GrLivArea"].mean() ))


In [ ]:
housing["Street"].value_counts()

In [ ]:
housing.isna().sum()

In [ ]:
housing.Condition1

In [ ]:
geocode_data.head()

In [ ]:
housing = pd.merge(housing, geocode_data.iloc[:,1:6], how='left', left_on='PID', right_on ="PID")
housing.isna().sum()

In [ ]:
#housing["exp_OverallQual"]= housing["OverallQual"].apply(lambda x: np.exp(x))
qual_related = housing.filter(regex='Qual$|Cond$').fillna("TA")

qual_related

In [ ]:
print(housing.groupby(['YrSold','MoSold'])['SalePrice'].aggregate(np.mean).plot(kind='bar'
                                                                              
                                                                             ))

In [ ]:
housing.SalePrice

In [ ]:
qual_related.GarageCond.value_counts()

In [ ]:
# Fill missing values vith TA
qual_related.fillna("TA", inplace=True)
def Rating(t):
    if t =="Ex": return  7
    elif t == "Gd": return 5
    elif t == "TA": return 3
    elif t == "Fa": return 2.5
    elif t == "Po": return 1
    else: return 0
for ele in qual_related.iloc[:,2:]:
    
    housing[ele]=qual_related[ele].map(Rating)
    qual_related[ele]=housing[ele]
housing

In [ ]:
housing.isna().sum()

In [ ]:
housing.SalePrice

In [ ]:
# Fill missing values vith TA

def yearidx(t):
    if t ==2006: return  1.186
    elif t == 2007: return 1.208
    elif t == 2008: return 1.198
    elif t == 2009: return 1.208
    elif t == 2010: return 1.19
    else: return 1.2

    
housing["GrLivArea_year"]=housing["YrSold"].map(yearidx)*housing["GrLivArea"]
housing

### UpSampling the Street labels

In [ ]:
housing.Street.value_counts()
def Ratio(t):
    if t == 'Pave': return 1.0
   
    else: return 180.0
# the returned values must be integers   
ratios = housing['Street'].map(Ratio)
index_repeat = housing.index.repeat(ratios)
index_repeat = pd.Series(index_repeat, name='repeat')
index_repeat.shape

In [ ]:
housing["Street"].value_counts()
housing["Street"] =housing["Street"].apply(lambda x: 1 if x=="Pave" else 0)
housing

housing = housing.loc[index_repeat].Street.value_counts()

In [ ]:
housing = pd.merge(index_repeat, housing, how='left', left_on = "repeat", right_on=housing.index)

In [ ]:
housing.SalePrice

In [ ]:
housing.isna().sum()

In [ ]:
def Ratio2(t):
    if t =="Normal": return  1.0
    elif t == "Partial": return 4
    elif t == "Alloca": return 5
    elif t == "Abnormal": return 6
    elif t == "Family": return 12
    else: return 100

ratios = housing['SaleCondition'].map(Ratio2)
index_repeat = housing.index.repeat(ratios)
index_repeat = pd.Series(index_repeat, name='repeat1')
index_repeat
housing = pd.merge(index_repeat, housing, how='left', left_on = "repeat1", right_on=housing.index)
housing.shape

In [ ]:
def fnd(t):
    if t =="PConc": return  1
    elif t == "CBlock": return 0
    else: return -0.5

housing['Foundation'] = housing['Foundation'].map(fnd)
    
    
       

In [ ]:
housing["GarageType"].value_counts()

In [ ]:
def grg(t):
    if t ==("Attchd" or "Builtin") : return  1
    else: return 0
    -0.5

housing['GarageType'] = housing['GarageType'].map(grg)

In [ ]:
housing.SalePrice

In [ ]:
housing["MasVnrType"] =np.where(housing["MasVnrType"]==("BrkCmn" or "Stone"),1,0)

In [ ]:
housing["RoofStyle"] = np.where(housing["RoofStyle"] == "Hip",1,0)

In [ ]:
# Checking unique PID #s
uni = housing.PID.unique()
uni.shape

In [ ]:
housing.isna().sum()

In [ ]:
housing.shape

In [ ]:
# How does the price vary by neighbourhood
plt.style.use('ggplot')



housing.boxplot(column ='SalePrice', by = 'Neighborhood')

In [ ]:
# merging 2.5Unf values into 1Story
housing['HouseStyle'].replace('2.5Unf', '1Story', inplace=True)
# merging 2.5Fin to 2Story
housing['HouseStyle'].replace('2.5Fin', '2Story', inplace=True)
# merging 1.5Unf values into 1.5Fin
housing['HouseStyle'].replace('1.5Unf', '1.5Fin', inplace=True)

In [ ]:
housing.HouseStyle.value_counts()

### Encoding and Dummyfication

In [ ]:
# get the averge price by neighborhood
dummy = housing.groupby(["Neighborhood","YrSold"])[["SalePrice"]].mean()
dummy.rename(columns = {"SalePrice":"Price_by_hood"}, inplace =True)
dummy

In [ ]:

housing = pd.merge(housing, dummy, how='left', on=['Neighborhood', 'Neighborhood'])
housing

In [ ]:
housing.isna().sum()

In [ ]:
housing.SaleType.value_counts()

In [ ]:
housing.columns

In [ ]:

# We trim the outliers from the list
#housing = housing
#leng = len(housing)
#print(leng)
#housing["Gradient"] = (housing.SalePrice-15000)/(housing.GrLivArea)

#housing=housing.sort_values(by="Gradient")[(housing.sort_values(by="Gradient")["Gradient"]>30) & (housing.sort_values(by="Gradient")["Gradient"]<220)]
#housing["Gradient2"] = (housing.SalePrice)/(housing.GrLivArea-1600.01) 
#housing=housing.sort_values(by="Gradient2")[ (housing.sort_values(by="Gradient2")["Gradient2"]>250)|(housing.sort_values(by="Gradient2")["Gradient2"]<0)]
#housing["Gradient3"] = (housing.SalePrice -100000)/(housing.TotalBsmtSF +1) 
#housing=housing.sort_values(by="Gradient3")[ (housing.sort_values(by="Gradient3")["Gradient3"]<200)]
#housing["Gradient4"] = (housing.SalePrice)/(housing.TotalBsmtSF-1200.01) 
#housing=housing.sort_values(by="Gradient4")[ (housing.sort_values(by="Gradient4")["Gradient4"]>300000/1300)|(housing.sort_values(by="Gradient4")["Gradient4"]<0)]

#housing
#leng2 = len(housing)
#outlier_pct = 100*(leng-leng2)/leng
#outlier_pct


In [ ]:
housing.PavedDrive = np.where(housing.PavedDrive=="N", 0,1)
housing.CentralAir = np.where(housing.CentralAir=="N", 0,1)

### Fixing Missing Values

In [ ]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_pct = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_pct], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns.sort_values(by= "Missing Values")[mis_val_table_ren_columns["Missing Values"]>10] 
    

In [ ]:
missing_values_table(housing)["Missing Values"].plot.bar()

In [ ]:
housing=housing.drop(["MiscFeature"], axis=1)

In [ ]:
housing[housing.columns[housing.isnull().any()]]

### "Proximity to various Conditions"

The columns *Condition 1* and *Condition 2* have the same realizations and can be regarded as "tags" given to a house indicating the nearby presence of a) a major street, b) a railroad, or c) a park.

The default tag "Norm" (implying no "condition") is given to 86% of the houses 

In [ ]:
housing["Condition1"] = housing["Condition1"].apply(lambda x: 0 if x =="Norm" else 1)
housing["Condition2"] = housing["Condition2"].apply(lambda x: 0 if x =="Norm" else 1)


In [ ]:
#changing the NA values which aren't NAs to different values to work better with the data set
medl = housing.LotFrontage.median()
medm = housing.MasVnrArea.median()
housing.Alley = housing.Alley.fillna("No Alley Access")
housing.LotFrontage = housing.LotFrontage.fillna(medl)
housing.MasVnrArea = housing.MasVnrArea.fillna(medm)
housing.MasVnrType = housing.MasVnrType.fillna('None')
housing.BsmtQual = housing.BsmtQual.fillna("No Basement")

In [ ]:

housing.PoolQC = housing.PoolQC.fillna(0)
housing.PoolQC=np.where(housing.PoolQC==0, 0,1)

In [ ]:
# Fill the NA with the right values
housing.BsmtCond = housing.BsmtCond.fillna("No Basement")
housing.BsmtExposure = housing.BsmtExposure.fillna("No Basement")
housing.BsmtFinType1 = housing.BsmtFinType1.fillna("No Basement")
housing.BsmtFinType2 = housing.BsmtFinType2.fillna("No Basement")
housing.FireplaceQu = housing.FireplaceQu.fillna("No Fireplace")
housing.GarageType = housing.GarageType.fillna("No Garage")
housing.GarageFinish = housing.GarageFinish.fillna("No Garage")
housing.GarageQual = housing.GarageQual.fillna("No Garage")
housing.GarageCond = housing.GarageCond.fillna("No Garage")
housing.Fence = housing.Fence.fillna("No Fence")
housing.Electrical = housing.Electrical.fillna("None")


In [ ]:
def ext(t):
    if t in  ['PreCast',  'ImStucc', 'CemntBd']:
        return 1
    elif t in ['AsphShn', 'AsbShng', 'CBlock']:
        return -1
    else: 
        return 0
housing['Exterior1st'] = housing['Exterior1st'].map(ext)
housing['Exterior2nd'] = housing['Exterior2nd'].map(ext)

for i in range(len(housing["Exterior1st"])):
    if housing["Exterior1st"][i] in  ['PreCast',  'ImStucc', 'CemntBd']:
        housing["Exterior1st"][i] =1
    elif housing["Exterior1st"][i] in ['AsphShn', 'AsbShng', 'CBlock']:
        housing["Exterior1st"][i] =-1
    else: 
        housing["Exterior1st"][i] =0
for i in range(len(housing["Exterior2nd"])):
    if housing["Exterior1st"][i] in  ['PreCast',  'ImStucc', 'CemntBd']:
        housing["Exterior1st"][i] =1
    elif housing["Exterior1st"][i] in ['AsphShn', 'AsbShng', 'CBlock']:
        housing["Exterior1st"][i] =-1
    else: 
        housing["Exterior1st"][i] =0    


In [ ]:
# Unify the two townhouse types into one.
housing["BldgType"] = housing["BldgType"].apply(
    lambda x: "Twnhs" if x in ("TwnhsE", "TwnhsI") else x
)
# Unify the two kinds of 2-family homes.
housing["BldgType"] = housing["BldgType"].apply(
    lambda x: "2Fam" if x in ("2FmCon", "Duplx") else x
)

In [ ]:

med1 = housing.BsmtFinSF1.median()
med2 = housing.BsmtFinSF2.median()
medf = housing.BsmtUnfSF.median()
medt = housing.TotalBsmtSF.median()
meda = housing.GarageArea.median()
medlon =housing.long.median()
medlat = housing.lat.median()
medist = housing.dist.median()
medinc = housing.income.median()

housing.BsmtFinSF1 = housing.BsmtFinSF1.fillna(med1)
housing.BsmtFinSF2 = housing.BsmtFinSF2.fillna(med2)
housing.BsmtUnfSF = housing.BsmtUnfSF.fillna(medf)
housing.TotalBsmtSF = housing.TotalBsmtSF.fillna(medt)
housing.GarageArea = housing.GarageArea.fillna(meda)
housing.long = housing.long.fillna(medlon)
housing.lat = housing.lat.fillna(medlat)
housing.dist = housing.dist.fillna(medist)
housing.income = housing.income.fillna(medinc)

housing.BsmtFullBath = housing.BsmtFullBath.fillna(0.0)
housing.BsmtHalfBath = housing.BsmtHalfBath.fillna(0.0)
housing.GarageCars = housing.GarageCars.fillna(0.0)

housing.GarageYrBlt = np.where(housing.GarageYrBlt.notnull(),housing.GarageYrBlt, housing.YearBuilt)

housing.TotalBsmtSF = np.where(housing.TotalBsmtSF<20,800, housing.TotalBsmtSF)
housing.BsmtFinSF1 = np.where(housing.BsmtFinSF1<20,800, housing.BsmtFinSF1)


In [ ]:
# We covert the year to age and take the squareroot to normalise the values
housing.YearBuilt = np.sqrt(2010 - housing.YearBuilt)
housing.GarageYrBlt = np.sqrt(2010 - housing.GarageYrBlt)
housing.YrSold = np.sqrt(2010 -housing.YrSold)
housing.YearRemodAdd = np.sqrt(2010 -housing.YearRemodAdd)

In [ ]:
housing.YearBuilt

In [ ]:
# Creation of new column combining full and half bathrooms into one
bathrm = (housing['FullBath'] + housing['BsmtFullBath'] +
(housing['HalfBath']*0.5) + (housing['BsmtHalfBath']*0.5))
housing['bathrm_cnt'] = bathrm

# Creation of new column combining deck/porch-related sq footage into one
patioSF = (housing['WoodDeckSF'] + housing['OpenPorchSF']+ housing['EnclosedPorch'] + 
           housing['3SsnPorch'] + housing['ScreenPorch'])
housing['patioSF'] = patioSF

In [ ]:
sns.distplot(housing.SalePrice, hist = False, kde = True,
            kde_kws = {'shade': True, 'linewidth': 2})
plt.show()

In [ ]:
housing["SalePrice"].mean()

In [ ]:
#use the log function to make the data normal
plt.hist(np.log(housing.SalePrice+1), bins = 50)

In [ ]:
sns.distplot(np.log(housing.SalePrice+1), hist = False, kde = True,
            kde_kws = {'shade': True, 'linewidth': 2})
plt.show()

In [ ]:
np.log(housing["SalePrice"]).mean()

In [ ]:
# Check the different types of foundations
#print(housing.Foundation.value_counts())
#sns.countplot(housing.Foundation)


In [ ]:
# Keep the numerical data to the left and categorical data to the right.
# Visualise the proportion of each categorical labels
categorical_data=[]
housing_new =pd.DataFrame()
#print(len(housing_new))
housing_new["SalePrice"]=housing["SalePrice"]
for ele in housing.columns:
    if np.dtype(housing[ele])== "int64" or np.dtype(housing[ele])=="float64":
        housing_new[ele] = housing[ele]
        print(len(housing_new))
    else:
        categorical_data.append(ele)
        sns.countplot(housing[ele])
    plt.show()

In [ ]:
for name in categorical_data:
    print(name, ': number of values', len(housing[name].value_counts()))

In [ ]:
for ele in categorical_data:
    housing_new[ele] = housing[ele]

In [ ]:
# Get the dummies of each categorical Data.
for ele in categorical_data:
    # Converting type of columns to category
    housing_new=pd.get_dummies(housing_new, prefix="{}_".format(ele), 
                            columns=[ele], 
                            drop_first=True)
    

housing_new#=housing_new.drop(["repeat","repeat_x", "repeat_y"], axis=1)

In [ ]:
housing

# Fit the data 

In [ ]:
from sklearn.model_selection import train_test_split
from itertools import combinations
 
# Get all combinations of according to the best contributions in increasing the R^2
# The process creates a partail ording among the predictors where A rel B if A is Subset of B and R2A <=R2B
# The time complexity of this process is n* nC1=n**2 where n is the number of columns
fnlst =[]
cscores =[]
for ele in range(1,len(housing_new.columns),1):
    
    comb = combinations(housing_new.iloc[:,1:].columns.difference(fnlst), 1)

    lst=[]
    scores=[]
    
    # Print the obtained combinations
    for i in list(comb):
        ols = LinearRegression()
        X_train, X_test, y_train, y_test = train_test_split(housing_new[fnlst+list(i)], housing_new.iloc[:,0], test_size=0.5, random_state=0)
        #lst.append(list(i))
        ols.fit(X_train, y_train)
        lst.append(list(i))
        scores.append(ols.score(X_train, y_train))
    cscores.append(pd.Series(scores).max())
    fnlst =fnlst+lst[pd.Series(scores)[pd.Series(scores)==pd.Series(scores).max()].index[0]]





ols = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(housing_new[fnlst], housing_new.iloc[:,0], test_size=0.5, random_state=0)

ols.fit(X_train, y_train)
print('-'*50)

print("R^2 for train set: %f" %ols.score(X_train, y_train))
print("R^2 for test set: %f" %ols.score(X_test, y_test))

print('-'*50)



plt.plot(range(len(cscores)), cscores)